In [1]:
#Importing Required Libraries

#For manipulating data as dataframes
import pandas as pd
import numpy as np

#for getting stock data using the yahoo finance API
import yfinance as yf

#for getting a list of all stock in the indian National Stock Exchange (NSE)
from nsetools import Nse

#for getting today's date
from datetime import datetime, timedelta, date

#For sending emails
import smtplib
import ssl
from email.message import EmailMessage
import os

In [2]:
#Creating an object for the Nse class belonging to nsetools module
nseObject = Nse()

#dictionary of all stock codes
all_stock_codes = nseObject.get_stock_codes()

#type(all_stock_codes)-------for verifying
#len(all_stock_codes)--------for verifying number of stocks

In [3]:
#creating a list of all stock codes in NSE
listStockCodes = list(all_stock_codes.keys())
#The first element is the label 'Symbol' so we remove that
listStockCodes.pop(0)

#getting today's date
now = datetime.now()

#putting date in required format
#today = date.today()
yday = now - timedelta(days=5)
#print(now)
#print(yday)

#creating dictionary
nse = {new_list: [] for new_list in listStockCodes}
    
print(datetime.now())

2022-08-04 14:38:49.290211


In [4]:
print(datetime.now())
for i in listStockCodes:
    try:
        ticker = yf.Ticker(i + '.NS')
        stockData = ticker.history(interval="1d", start=yday, end=now)
        closetoday = stockData.iloc[-1]['Close']
        closeyday = stockData.iloc[-2]['Close']
        closepriceGrowth = (closetoday - closeyday)/closeyday*100
        tradeVolumetoday = stockData.iloc[-1]['Volume']
        tradeVolumeyday = stockData.iloc[-2]['Volume']
        volumeGrowth = (tradeVolumetoday - tradeVolumeyday)/tradeVolumeyday * 100
        L = [closetoday, closeyday, closepriceGrowth, tradeVolumetoday, tradeVolumeyday, volumeGrowth]
        nse[i] = L
    except IndexError:
        nse.pop(i)
        continue

2022-08-04 14:38:53.103421


C:\Users\dhing\AppData\Local\Temp/ipykernel_976/4092151396.py:11: RuntimeWarning: invalid value encountered in double_scalars
  volumeGrowth = (tradeVolumetoday - tradeVolumeyday)/tradeVolumeyday * 100


- KALYANI.NS: No data found for this date range, symbol may be delisted


C:\Users\dhing\AppData\Local\Temp/ipykernel_976/4092151396.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  volumeGrowth = (tradeVolumetoday - tradeVolumeyday)/tradeVolumeyday * 100


- NIRAJISPAT.NS: No data found for this date range, symbol may be delisted
- RAMAPHO.NS: No data found, symbol may be delisted
- ZODIAC.NS: No data found for this date range, symbol may be delisted


In [6]:
stocks = pd.DataFrame.from_dict(nse)
    
stocks2 = stocks.transpose()
stocks2.rename(columns = {0:'Close Price Today', 1:'Close Price Yesterday',2:'Close Price Growth', 3:'Trade Volume Today', 4: 'Trade Volume Day Yesterday', 5: 'Trade Volume Growth'}, inplace = True)
stocks2

,Close Price Today,Close Price Yesterday,Close Price Growth,Trade Volume Today,Trade Volume Day Yesterday,Trade Volume Growth
20MICRONS,98.199997,98.050003,0.152977,488197.0,367226.0,32.941840
21STCENMGM,24.200001,23.850000,1.467507,4464.0,3603.0,23.896753
3IINFOLTD,40.000000,40.750000,-1.840491,620276.0,613524.0,1.100527
3MINDIA,23472.500000,22790.099609,2.994284,3747.0,2103.0,78.174037
3PLAND,15.050000,15.250000,-1.311474,4969.0,653.0,660.949464
...,...,...,...,...,...,...
ZOTA,272.100006,266.549988,2.082168,6219.0,17190.0,-63.821990
ZUARI,158.750000,154.250000,2.917342,1053290.0,160430.0,556.541794
ZUARIIND,162.300003,163.000000,-0.429446,59697.0,25405.0,134.981303
ZYDUSLIFE,362.500000,346.450012,4.632699,3583959.0,866092.0,313.808117


In [7]:
stocks2 = stocks2.sort_values(by = "Close Price Growth", ascending = False)
stocksT = stocks2.sort_values(by = "Trade Volume Growth", ascending = False)

In [12]:
 #putting the data in html format
html = r"<html>"+"\n"
title = "\t"+r"<p>Top 30 according to Trade Volume</p><br>"+"\n"
html = html + title + "\t\t"+r"<table>"+"\n"
html = html + "\t\t\t"+r"<tr>"+"\n" 
html = html + "\t\t\t\t"+r"<th>" + "Symbol" + r"</th>"+"\n"
for i in range(3,6):
    html = html + "\t\t\t\t"+r"<th>" + stocksT.columns[i] + r"</th>"+"\n"
    html = html + "\t\t\t"+r"</tr>"+"\n" 
for i in range(20):
    temp = "\t\t\t"+r"<tr>"+"\n"
    temp = temp + "\t\t\t\t"+r"<td>" + stocksT.index[i] + r"</td>"+"\n"
    for k in range(3,6):
        temp = temp + "\t\t\t\t"+r"<td>" + str(stocksT.iloc[i][k]) + r"</td>"+"\n"
    temp = temp + "\t\t\t"+r"</tr>"+"\n"
    html = html + temp
html = html + "\t\t"+r"</table>"+"\n"

html = html + r"<br><br><br>"

title = "\t"+r"<p>Top 30 according to close price change</p><br>"+"\n"
html = html + title + "\t\t"+r"<table>"+"\n"
html = html + "\t\t\t"+r"<tr>"+"\n" 
html = html + "\t\t\t\t"+r"<th>" + "Symbol" + r"</th>"+"\n"
for i in range(3,6):
    html = html + "\t\t\t\t"+r"<th>" + stocks2.columns[i] + r"</th>"+"\n"
html = html + "\t\t\t"+r"</tr>"+"\n" 
for i in range(20):
    temp = "\t\t\t"+r"<tr>"+"\n"
    temp = temp + "\t\t\t\t"+r"<td>" + stocks2.index[i] + r"</td>"+"\n"
    for k in range(3,6):
        temp = temp + "\t\t\t\t"+r"<td>" + str(stocks2.iloc[i][k]) + r"</td>"+"\n"
    temp = temp + "\t\t\t"+r"</tr>"+"\n"
    html = html + temp
html = html + "\t\t"+r"</table>"+"\n"
html = html + r"</html>"

In [13]:
SENDER = 'dailystocks2022@gmail.com'
    
PASS = os.environ.get('EmailPassword')
    
TO = ['dhingra.brinda@gmail.com','sanjay.dhingra@gmail.com','avantikasuryawanshi@gmail.com']
    
subject = 'Change in Trade Volumes for '+str(now.day)+'.'+str(now.month)+'.'+str(now.year)
    
em = EmailMessage()
em['From'] = SENDER
em['To'] = TO
em['Subject'] = subject
em.set_content(html,'html')
context = ssl.create_default_context()
with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
    smtp.login(SENDER, PASS)
    smtp.sendmail(SENDER, TO, em.as_string())

In [15]:
stocksT[:30]

,Close Price Today,Close Price Yesterday,Close Price Growth,Trade Volume Today,Trade Volume Day Yesterday,Trade Volume Growth
SAKAR,197.899994,56.900002,247.803143,10171.0,0.0,inf
NORBTEAEXP,6.400000,6.100000,4.918036,230.0,0.0,inf
KSHITIJPOL,41.000000,32.450001,26.348225,654657.0,0.0,inf
RANEENGINE,251.949997,230.000000,9.543477,70618.0,700.0,9988.285714
ABMINTLLTD,78.800003,75.650002,4.163915,925.0,25.0,3600.000000
LODHA,1041.000000,1104.849976,-5.779063,7697272.0,240364.0,3102.339785
MOHITIND,18.600000,16.000000,16.250002,500850.0,21218.0,2260.495805
SOUTHWEST,197.399994,189.699997,4.059039,42935.0,1878.0,2186.208733
SITINET,1.800000,1.750000,2.857140,4344887.0,202314.0,2047.595816
RUBYMILLS,453.850006,403.200012,12.562002,325906.0,15983.0,1939.079021


In [16]:
stocks2[:30]

,Close Price Today,Close Price Yesterday,Close Price Growth,Trade Volume Today,Trade Volume Day Yesterday,Trade Volume Growth
SAKAR,197.899994,56.900002,247.803143,10171.0,0.0,inf
KSHITIJPOL,41.000000,32.450001,26.348225,654657.0,0.0,inf
SUBEXLTD,39.950001,33.299999,19.969975,26656201.0,9553145.0,179.030633
PARACABLES,13.100000,10.950000,19.634709,674051.0,111930.0,502.207630
SUVIDHAA,8.000000,6.700000,19.402988,1309322.0,80261.0,1531.330285
MOHITIND,18.600000,16.000000,16.250002,500850.0,21218.0,2260.495805
HLEGLAS,3545.000000,3116.949951,13.732978,317311.0,15730.0,1917.234584
TAKE,28.450001,25.049999,13.572861,3842397.0,231995.0,1556.241298
RUBYMILLS,453.850006,403.200012,12.562002,325906.0,15983.0,1939.079021
NAVNETEDUL,131.050003,117.800003,11.247877,8356593.0,9101007.0,-8.179468
